In [18]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# Import the essentials
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import OrdinalEncoder
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
import sys

# Load the trained models
passenger_model = joblib.load('/content/drive/MyDrive/ML PROJECT/Bus_Service_Predictions/Resultant Models/passenger_count_prediction_model.pkl')
price_model = joblib.load('/content/drive/MyDrive/ML PROJECT/Bus_Service_Predictions/Resultant Models/price_prediction_model.pkl')

# Load dataset for reference values
merged_df = pd.read_csv('/content/drive/MyDrive/ML PROJECT/Bus_Service_Predictions/Data Preprocessing/merged_bus_data.csv')

# Define categorical columns used for encoding
categorical_cols = ['From_Location', 'To_Location', 'Time_Slot', 'Day_Type', 'Special_Event']

# Initialize and fit OrdinalEncoder on actual merged data
ordinal_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
ordinal_enc.fit(merged_df[categorical_cols].astype(str))

# Grab unique locations
all_from_locations = merged_df['From_Location'].unique()
all_to_locations = merged_df['To_Location'].unique()

# Create output containers for widgets
passenger_output = widgets.Output()
price_output = widgets.Output()
kpi_output = widgets.Output()
route_check_output = widgets.Output()


# Display Key Performance Indicators (KPIs)
def show_kpis():
    with kpi_output:
        kpi_output.clear_output()

        # Core metrics
        total_revenue = merged_df['Total_Revenue'].sum()
        passenger_count = merged_df['Passenger_Count'].sum()
        total_journeys = len(merged_df)
        total_distance = merged_df['Distance_km'].sum()

        # Derived metrics
        avg_passengers_per_journey = passenger_count / total_journeys if total_journeys else 0
        avg_revenue_per_km = total_revenue / total_distance if total_distance else 0

        # Top performing route by revenue
        route_revenue = merged_df.groupby(['From_Location', 'To_Location'])['Total_Revenue'].sum()
        top_revenue_route = route_revenue.idxmax()
        top_revenue_value = route_revenue.max()

        # Most frequent route
        route_counts = merged_df.groupby(['From_Location', 'To_Location']).size()
        most_frequent_route = route_counts.idxmax()
        most_frequent_count = route_counts.max()

        # Display KPIs
        print("=== Key Performance Indicators ===")
        print(f"✔ Total Revenue: ₹{total_revenue:,.2f}")
        print(f"✔ Total Passenger Count: {int(passenger_count)}")
        print(f"✔ Total Journeys: {total_journeys}")
        print(f"✔ Average Passengers per Journey: {avg_passengers_per_journey:.2f}")
        print(f"✔ Top Performing Route (by Revenue): {top_revenue_route[0]} ➝ {top_revenue_route[1]} (₹{top_revenue_value:,.2f})")
        print(f"✔ Most Frequent Route: {most_frequent_route[0]} ➝ {most_frequent_route[1]} ({most_frequent_count} journeys)")
        print(f"✔ Average Revenue per Kilometer across all routes: ₹{avg_revenue_per_km:,.2f}/km")


# Route checking logic
def on_check_route_clicked(b):
    from_loc = from_dropdown.value
    to_loc = to_dropdown.value

    with route_check_output:
        route_check_output.clear_output()
        is_available = ((merged_df['From_Location'] == from_loc) &
                        (merged_df['To_Location'] == to_loc)).any()
        if is_available:
            print(f"Route from {from_loc} to {to_loc} is available.")
        else:
            print(f"Route from {from_loc} to {to_loc} is NOT available in the dataset.")

# Create a distance lookup dictionary for quick access
distance_lookup = merged_df.groupby(['From_Location', 'To_Location'])['Distance_km'].mean().to_dict()

# Get distance between two locations (handles both directions)
def get_distance(from_loc, to_loc):
    try:
        return distance_lookup[(from_loc, to_loc)]
    except KeyError:
        try:
            return distance_lookup[(to_loc, from_loc)]
        except KeyError:
            return None

def passenger_prediction(from_location, to_location, time_slot, day_type, weekday_num, special_event, discount):
    with passenger_output:
        clear_output()

        # Check for missing inputs
        if None in [time_slot, day_type, special_event, from_location, to_location, weekday_num]:
            print("Please select all dropdown values.")
            return

        try:
            high_demand = 0
            distance_km = get_distance(from_location, to_location)
            if distance_km is None:
                print(f"Distance between {from_location} and {to_location} not found.")
                return

            # Prepare input dataframe for prediction
            input_df = pd.DataFrame({
                'Distance_km': [distance_km],
                'Discount_Offered (%)': [discount],
                # 'Total_Revenue': [revenue],
                'High_Demand': [high_demand],
                'Time_Slot': [time_slot],
                'Day_Type': [day_type],
                'Special_Event': [special_event],
                'Day': [1],
                'Month': [1],
                'Weekday_Num': [weekday_num],
                'From_Location': [from_location],
                'To_Location': [to_location],
                # 'Ticket_Price': [ticket_price]
            })

            # Encode categorical features
            categorical_cols = ['From_Location', 'To_Location', 'Time_Slot', 'Day_Type', 'Special_Event']
            try:
                # Convert to string and encode
                input_df[categorical_cols] = input_df[categorical_cols].astype(str)
                encoded_values = ordinal_enc.transform(input_df[categorical_cols])
                for i, col in enumerate(categorical_cols):
                    input_df[f"{col}_Encoded"] = encoded_values[:, i]
            except Exception as e:
                print(f"Error during encoding: {e}")
                return

            # Create all expected columns
            expected_cols = passenger_model.feature_names_in_
            for col in expected_cols:
                if col not in input_df.columns:
                    if col == 'Weekday_Encoded':
                        input_df[col] = weekday_num
                    elif col in ['Day', 'Month']:
                        input_df[col] = 1  # Default values
                    else:
                        input_df[col] = 0  # Default for other missing columns

            # Ensure columns match training data
            input_df = input_df[expected_cols]

            # Prediction step
            prediction = passenger_model.predict(input_df)[0]
            print(f"Predicted Passenger Count: {int(prediction)}")

        except Exception as e:
            print(f"Error during prediction: {e}")

def price_prediction(from_location, to_location, time_slot, day_type, weekday_num, special_event,
                     discount, high_demand, date_str):
    with price_output:
        clear_output()

        # Check all required inputs
        if None in [from_location, to_location, time_slot, day_type, special_event, high_demand, weekday_num, date_str]:
            print("Please select all required fields.")
            return

        try:
            distance_km = get_distance(from_location, to_location)
            if distance_km is None:
                print(f"Distance between {from_location} and {to_location} not found.")
                return

            # Parse date
            date = datetime.strptime(date_str, '%Y-%m-%d')
            day = date.day
            month = date.month

            # Prepare input dataframe
            input_df = pd.DataFrame({
                # 'Passenger_Count': [passenger_count],
                'Distance_km': [distance_km],
                'Discount_Offered (%)': [discount],
                # 'Total_Revenue': [revenue],
                'High_Demand': [high_demand],
                'Time_Slot': [time_slot],
                'Day_Type': [day_type],
                'Special_Event': [special_event],
                'Day': [day],
                'Month': [month],
                'Weekday_Num': [weekday_num],
                'From_Location': [from_location],
                'To_Location': [to_location]
            })

            # Encode categorical features
            categorical_cols = ['From_Location', 'To_Location', 'Time_Slot', 'Day_Type', 'Special_Event']
            try:
                # Convert to string and encode
                input_df[categorical_cols] = input_df[categorical_cols].astype(str)
                encoded_values = ordinal_enc.transform(input_df[categorical_cols])
                for i, col in enumerate(categorical_cols):
                    input_df[f"{col}_Encoded"] = encoded_values[:, i]
            except Exception as e:
                print(f"Error during encoding: {e}")
                return

            # Create all expected columns
            expected_cols = price_model.feature_names_in_
            for col in expected_cols:
                if col not in input_df.columns:
                    if col == 'Weekday_Encoded':
                        input_df[col] = weekday_num
                    elif col in ['Day', 'Month']:
                        pass  # Already set
                    else:
                        input_df[col] = 0  # Default for other missing columns

            # Ensure columns match training data
            input_df = input_df[expected_cols]

            # Predict ticket price
            prediction = price_model.predict(input_df)[0]
            print(f"Predicted Ticket Price: ₹{round(prediction, 2)}")

        except Exception as e:
            print(f"Error during prediction: {e}")

# --- Widgets Setup ---

# KPIs Button
kpi_button = widgets.Button(description="Show KPIs")

# Passenger Prediction Widgets
passenger_time_slot = widgets.Dropdown(
    options=[('Select Timeslot', None), ('Morning', 'Morning'), ('Afternoon', 'Afternoon'), ('Evening', 'Evening'), ('Night', 'Night')],
    description='Time Slot:', style={'description_width': '100px'}
)
passenger_day_type = widgets.Dropdown(
    options=[('Select Day Type', None), ('Weekday', 'Weekday'), ('Weekend', 'Weekend')],
    description='Day Type:', style={'description_width': '100px'}
)
passenger_special_event = widgets.Dropdown(
    options=[
        ('Select Special Event', None),
        ('Festival', 'Festival'),
        ('Holiday', 'Holiday'),
        ('Weekend', 'Weekend'),
        ('No Event', 'No Event')
    ],
    description='Special Event:', style={'description_width': '100px'}
)

passenger_discount = widgets.BoundedFloatText(
    value=0, min=0, max=100, description='Discount (%):', style={'description_width': '100px'}
)

passenger_from_location = widgets.Dropdown(
    options=[('Select From', None)] + [(loc, loc) for loc in all_from_locations],
    description='From:', style={'description_width': '100px'}
)
passenger_to_location = widgets.Dropdown(
    options=[('Select To', None)] + [(loc, loc) for loc in all_to_locations],
    description='To:', style={'description_width': '100px'}
)
passenger_weekday = widgets.Dropdown(
    options=[('Select Weekday', None), ('Monday', 0), ('Tuesday', 1), ('Wednesday', 2),
             ('Thursday', 3), ('Friday', 4), ('Saturday', 5), ('Sunday', 6)],
    description='Weekday:', style={'description_width': '100px'}
)
passenger_button = widgets.Button(
    description="Predict Passenger Count", layout=widgets.Layout(width='250px', height='25px')
)

# Price Prediction Widgets
price_from_location = widgets.Dropdown(
    options=[('Select From', None)] + [(loc, loc) for loc in all_from_locations],
    description='From:', style={'description_width': '100px'}
)
price_to_location = widgets.Dropdown(
    options=[('Select To', None)] + [(loc, loc) for loc in all_to_locations],
    description='To:', style={'description_width': '100px'}
)
price_time_slot = widgets.Dropdown(
    options=[('Select Timeslot', None), ('Morning', 'Morning'), ('Afternoon', 'Afternoon'), ('Evening', 'Evening'), ('Night', 'Night')],
    description='Time Slot:', style={'description_width': '100px'}
)
price_day_type = widgets.Dropdown(
    options=[('Select Day Type', None), ('Weekday', 'Weekday'), ('Weekend', 'Weekend')],
    description='Day Type:', style={'description_width': '100px'}
)
price_special_event = widgets.Dropdown(
    options=[
        ('Select Special Event', None),
        ('Festival', 'Festival'),
        ('Holiday', 'Holiday'),
        ('Weekend', 'Weekend'),
        ('No Event', 'No Event')
    ],
    description='Special Event:', style={'description_width': '100px'}
)

price_discount = widgets.BoundedFloatText(
    value=0, min=0, max=100, description='Discount (%):', style={'description_width': '100px'}
)

price_high_demand = widgets.Dropdown(
    options=[('Select Demand', None), ('Yes', 1), ('No', 0)],
    description='High Demand:', style={'description_width': '100px'}
)
price_weekday = widgets.Dropdown(
    options=[('Select Weekday', None), ('Monday', 0), ('Tuesday', 1), ('Wednesday', 2),
             ('Thursday', 3), ('Friday', 4), ('Saturday', 5), ('Sunday', 6)],
    description='Weekday:', style={'description_width': '100px'}
)
price_date = widgets.DatePicker(
    description='Journey Date:', style={'description_width': '100px'}
)
price_button = widgets.Button(
    description="Predict Ticket Price", layout=widgets.Layout(width='250px', height='25px')
)

# Routes Tab Button
routes_button = widgets.Button(
    description="Show Available Routes", layout=widgets.Layout(width='250px', height='25px')
)

# Dropdown widgets for input
from_dropdown = widgets.Dropdown(
    options=[('Select From', None)] + [(loc, loc) for loc in sorted(all_from_locations)],
    description='From:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='300px')
)

to_dropdown = widgets.Dropdown(
    options=[('Select To', None)] + [(loc, loc) for loc in sorted(all_to_locations)],
    description='To:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='300px')
)

# Button to check route
check_route_button = widgets.Button(
    description="Check Route Availability",
    layout=widgets.Layout(width='300px', height='35px')
)

# --- Event handlers ---

def on_kpi_button_clicked(b):
    show_kpis()

def on_passenger_button_clicked(b):
    passenger_prediction(
        passenger_from_location.value,
        passenger_to_location.value,
        passenger_time_slot.value,
        passenger_day_type.value,
        passenger_weekday.value,
        passenger_special_event.value,
        passenger_discount.value,
    )

def on_price_button_clicked(b):
    price_prediction(
        price_from_location.value,
        price_to_location.value,
        price_time_slot.value,
        price_day_type.value,
        price_weekday.value,
        price_special_event.value,
        price_discount.value,
        price_high_demand.value,
        price_date.value.strftime('%Y-%m-%d') if price_date.value else ''
    )



# Link buttons with functions
kpi_button.on_click(on_kpi_button_clicked)
passenger_button.on_click(on_passenger_button_clicked)
price_button.on_click(on_price_button_clicked)
check_route_button.on_click(on_check_route_clicked)

# --- Layout & Tabs ---

tab = widgets.Tab()

passenger_tab = widgets.VBox([
    passenger_from_location,
    passenger_to_location,
    passenger_time_slot,
    passenger_day_type,
    passenger_weekday,
    passenger_special_event,
    passenger_discount,
    passenger_button,
    passenger_output
])

price_tab = widgets.VBox([
    price_from_location,
    price_to_location,
    price_time_slot,
    price_day_type,
    price_weekday,
    price_special_event,
    price_discount,
    price_high_demand,
    price_date,
    price_button,
    price_output
])

kpi_tab = widgets.VBox([
    kpi_button,
    kpi_output
])


routes_tab = widgets.VBox([
    widgets.Label("Select locations to check route availability:"),
    from_dropdown,
    to_dropdown,
    check_route_button,
    route_check_output
])

tab.children = [kpi_tab, routes_tab, passenger_tab, price_tab]
tab.set_title(0, 'KPIs')
tab.set_title(1, 'Check Route')
tab.set_title(2, 'Passenger Prediction')
tab.set_title(3, 'Price Prediction')

display(tab)